🔧 Piperack Creator ──➤ 🏗️  Piperack Generator  ──➤ 🔀⚡ ***Piperack Optimiser***

# Piperack Optimiser 1.0

> Segregate different elements and apply profiles as required with integrated profile applicator

In [1]:
from base import *
from base.piperack.load import *
from base.staad_base.root import run_analysis

In [ ]:
# Object creation with provided data
PR = Piperack('MPR_C-C')
PR = PR.load_from_pickle(filename=fr'piperack_objects\{PR.name}.pkl')

PR_S = PR.generate_structural_components(geometry=geometry,property=property)

beam_groups = [
    BeamGroup(heading='Columns', item_dict=PR_S.columns_y),
    BeamGroup(heading='Stub Columns', item_dict=PR_S.stub_columns_y),
    BeamGroup(heading='Tier Beams', item_dict={tier.base.y: tier.beams for tier in PR.tiers}),
    BeamGroup(heading='Tier Int Beams', item_dict={tier.base.y: tier.int_beams for tier in PR.tiers}),
    BeamGroup(heading='VB', item_dict=group_beams_by_y(PR_S.vertical_braces)),
    BeamGroup(heading='PB', item_dict=group_beams_by_y(PR_S.plan_braces)),
    BeamGroup(heading='Long Beams', item_dict=group_beams_by_y(PR_S.long_beams)),
    BeamGroup(heading='Long Int Beams', item_dict=group_beams_by_y(PR_S.int_long_beams)),
    BeamGroup(heading='Bracket Beams', item_dict=group_beams_by_y(PR_S.bracket_beams)),
    BeamGroup(heading='Bracket Braces', item_dict=group_beams_by_y(PR_S.bracket_braces))
]

bb_selector = ButtonBar().set_selector(select_beams)
button_bars = [bb_selector.set_heading(bg.heading).set_item_dict(bg.item_dict) for bg in beam_groups]

display(HTML(dark_theme_css))
display(widgets.HBox( [button.create_button_bar_for_dict() for button in button_bars] , layout=dark_hbox))

staad_profiles = get_staad_profiles()
staad_section_ref_nos = {profile['name']:profile for profile in staad_profiles}

staad_profiles_widget = create_profile_widget(staad_profiles)
steel_widget, steel_dropdown = create_steel_section_widget(steel_sections)

profile_button_bar = widgets.VBox([create_button(label='Insert Profile',predicate=lambda:insert_profile_button_click(get_section_ref_no=get_section_ref_no,
                                                                                                                     steel_dropdown=steel_dropdown,
                                                                                                                     staad_section_ref_nos=staad_section_ref_nos,
                                                                                                                     )), 
                                  
                                   create_button(label='Apply Profile',predicate=lambda:apply_profile_button_click(get_section_ref_no=get_section_ref_no,
                                                                                                                   steel_dropdown=steel_dropdown,
                                                                                                                   staad_section_ref_nos=staad_section_ref_nos,
                                                                                                                   geometry=geometry,
                                                                                                                   property=property,
                                                                                                                   get_selected_beam_nos=get_selected_beam_nos,
                                                                                                                   beam_list_copy_and_display=beam_list_copy_and_display,
                                                                                                                   assign_profile=assign_profile,
                                                                                                                   assign_material=assign_material))])

display(widgets.HBox([staad_profiles_widget,steel_widget, profile_button_bar]))

In [3]:
# run_analysis(openSTAAD)
    
beam_objects: dict = get_beam_objects(geometry=geometry, property=property)
beam_nos: list = list(beam_objects.keys())

all_members = member_group(
    id='whole_structure',
    members=beam_nos,
    Staad_objects=STAAD_objects,
    allowable_ratio=0.85
)
all_members.get_utilization_ratios()

# Display the HTML output
display(HTML(dark_theme_css))
display(HTML(generate_model_results_html(all_members=all_members,step=0.25)))